# Simulator

## Classification

In [2]:
from classification import get_class_questions
from classification import get_class_question

def classification():
  questions = get_class_questions()
  filtered_questions = get_class_question("Easy", questions, "music")

  for index, row in filtered_questions.iterrows():
    print(row['question'])    

In [3]:
classification()

At what age did Jimmy Hendrix, Janis Joplin and Kurt Cobain die?
Freddie Mercury was the lead vocalist of which rock band?
From which country did The Beatles come from?
How many strings does a guitar have?
In what band was John Lennon vocalist?
Miley Cyrus is the daughter of country singer Billy Ray Cyrus?
SpongeBob SquarePants has a best friend, what is his name? - "Mr. Krabs"? "Patrick Star"? "Squidward tentacles"? "David Hasselhoff"?'
True or False? A British man had his name legally changed to PlayStation.
What instrument did Jimi Hendrix play?
What instrument does a pianist play?
What is the continuation of the song by Carly Rae Jepsen?; Hey, I just met you and this is crazy But here's my number, __ ____ __ _____
What is the most played song on youtube?
What is the name of Cinderella's prince?
What is the name of the band behind "Barbie Girl"?
What is the name of the largest music festival in Denmark
What is the name of the toy cowboy in Toy Story?
What kind of music does Skrillex

## Convergence

In [7]:
import random

from convergence import calc_threshold
from convergence import get_answers_to_questions
from convergence import get_next_question
from convergence import get_conv_questions
from convergence import give_answer
from convergence import get_question_answer

def decision(probability=0.9):
  return random.random() < probability

def polymath_user(probability):
  skipped = []
  answers_to_questions = get_answers_to_questions()
  questions = get_conv_questions()
  not_finished = True
  amount_of_questions = 0

  while not_finished:
    for n,g in questions:
      if n not in skipped:
        if calc_threshold(0, n):
          skipped.append(n)
          continue
        question = get_next_question(g)
        answer_generated = question["answer"].to_string(index=False)
        question_string = question.to_string(index=False)

        answer = get_question_answer(answers_to_questions,question_string)
        # hope for the best that the answers are right or available
        if len(skipped) <= 4:
          amount_of_questions += 1
          if decision(probability):
            if not answer_generated:
              user_answer = answer
              give_answer(answer, answer_generated, user_answer, n)
            else:
              user_answer = answer_generated
              give_answer(answer, answer_generated, user_answer, n)
          else:
            give_answer(answer, answer_generated, "False Answer", n)
        if len(skipped) == 4:
          print("Amount of questions answered:", amount_of_questions)
          print("Finished topic was:", n)
          not_finished = False


def topic_expert_user(probability, topic):
  skipped = []
  answers_to_questions = get_answers_to_questions()
  questions = get_conv_questions()
  not_finished = True
  amount_of_questions = 0

  while not_finished:
    for n,g in questions:
      if n not in skipped:
        if calc_threshold(-1, n):
          skipped.append(n)
          continue
        question = get_next_question(g)
        answer_generated = question["answer"].to_string(index=False)
        question_string = question.to_string(index=False)

        answer = get_question_answer(answers_to_questions,question_string)
        # hope for the best that the answers are right or available
        if n == topic:
          if len(skipped) <= 4:
            if decision(probability):
              amount_of_questions +=1
              if not answer_generated:
                user_answer = answer
                give_answer(answer, answer_generated, user_answer, n)
              else:
                user_answer = answer_generated
                give_answer(answer, answer_generated, user_answer, n)
            else:
              give_answer(answer, answer_generated, "False Answer", n)
        else:
          give_answer(answer, answer_generated, "False Answer", n)
        if len(skipped) == 4:
          print("Amount of questions answered:", amount_of_questions)
          print("Finished Topic was:", n)
          not_finished = False

In [8]:
polymath_user(0.5)
topic_expert_user(0.5, "music")

KeyboardInterrupt: 

## Friend Recommender

In [15]:
import nltk
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.feature_extraction.text import TfidfVectorizer

def takeCorrelation(elem):
  return elem[1]

STEMMER = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens, stemmer=STEMMER):
  return [stemmer.stem(item) for item in tokens]


def tokenizer(text):
  tokens = nltk.word_tokenize(text)
  return stem_tokens(tokens)

def euclidean_distance(x, y):
  return np.sqrt(np.sum((x - y) ** 2))


def get_recommendations_feature(cutoff_k, questions, user_id):
  print("Feature based recommendation")
  tfidf = TfidfVectorizer(tokenizer=tokenizer, stop_words='english')
  tfs = tfidf.fit_transform(questions['question'])
  # add column for vector
  questions['tfsvector'] = list(tfs.toarray())
  user_features = questions[questions['id'] == int(user_id)]['tfsvector'].mean()
  # the questions which a user likes
  questions = questions.groupby(['id'])
  friends_questions = []
  for n, g in questions:
    g = g[g['opinion'] > 1]['tfsvector'].mean()
    friends_questions.append((n, euclidean_distance(user_features, g)))
  friends_questions.sort(key=takeCorrelation, reverse=True)
  for friend in friends_questions[:int(cutoff_k)]:
    print(friend)


def get_recommendations_topic(cutoff_k, questions, user_id):
  friends = []
  print("Topic based recommendation:")
  user_features = questions[questions['id'] == int(user_id)].groupby(['topic'])[
    'opinion'].mean()
  questions = questions.groupby(['id'])
  for n, g in questions:
    if n == int(user_id):
      continue
    # when classification not works properly then some users have only 4 topics
    if len(g.groupby(['topic'])['opinion'].mean()) < 5:
      continue
    corr, p_value = pearsonr(user_features,
                             g.groupby(['topic'])['opinion'].mean())
    friends.append((n, corr))
  friends.sort(key=takeCorrelation, reverse=True)
  for friend in friends[:int(cutoff_k)]:
    print(friend)

In [16]:
# cutoff, questions, user_id
get_recommendations_feature(5,get_friends_questions(),3)

Feature based recommendation


/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

(4, 0.36076689057979405)
(11, 0.3386440771631049)
(27, 0.3282917165082846)
(24, 0.30563175815672294)
(31, 0.297454486813683)


In [17]:
# cutoff, questions, user_id
get_recommendations_topic(5,get_friends_questions(),4)

Topic based recommendation:
(17, 0.7857949147736706)
(14, 0.684028654311571)
(32, 0.6347061932859925)
(34, 0.5637752809434675)
(19, 0.5146411253002409)


## User Preference

In [23]:
def get_users_questions():
  questions = pd.read_csv("data/classified.csv",
                          encoding="utf-8", sep=",")
  return questions



def get_preferance_id(questions, userid):
  user_questions = questions[questions['id'] == int(userid)]
  return user_questions


def get_prefereance_filtered_qustions(questions, user_preferance):
  questions = questions[questions['topic'] == user_preferance]
  return questions


def get_user_preferance(user_questions):
  user_questions = user_questions.groupby('topic')['opinion'].mean()
  user_preferance = user_questions.idxmax()
  return user_preferance

In [25]:
user_questions = get_preferance_id(get_users_questions(),5)
user_preferance = get_user_preferance(user_questions)
get_prefereance_filtered_qustions(get_users_questions(),user_preferance)

,id,question,answer,difficulty,opinion,factuality,topic
0,0,Who invented the light bulb?,Thomas Edison,Medium,2,0,science-technology
1,0,Who is the only woman to have ever received two Nobel prizes?,Marie Curie,Hard,2,0,science-technology
2,0,What's the most expensive material on Earth?,dont know,Hard,2,0,science-technology
6,0,Who lives in a pineapple under the sea?,Spongebob Squarepants,Easy,3,0,science-technology
7,0,What's the name of the reindeer in Frozen?,Sven,Medium,3,0,science-technology
8,0,What is the top color in a rainbow?,red,Hard,3,0,science-technology
14,0,In what year was Space Invaders released?,1978,Hard,3,0,science-technology
15,0,What is Linux's creator's full name?,Linus Torvalds,Medium,3,0,science-technology
16,0,Which country in the world has maximum number of robots working?,Korea,Hard,1,0,science-technology
17,0,A type glass that is highly resistant to heat?,Heat resistant glass,Hard,1,1,science-technology


## Difficulty